# Introduction au Dataframe avec Spark


## 1 Présentation

Pour faciliter le développement de programme, Spark à partir de la version 2 intégre la notion de Dataframe.

Le DataFrame est une couche d'abstraction des RDD, qui présentent les données comme des tables de base de données sans se préoccuper de la taille des données.
Les DataFrames offrent de nombreux avantages:
* Une syntaxe beaucoup plus simple.
* Possibilité d'utiliser SQL directement dans la trame de données.
* La parallélisation des traitements dans une architecture distribuée est gérée par Spark.
    
Si vous avez utilisé R ou même la bibliothèque pandas avec Python, vous connaissez probablement déjà le concept des DataFrames. Même si Spark supporte plusieurs langages il faut savoir que le langage le moins efficace est le python.

Vous êtes prêt, jouons un peu avec les dataframes.

Let's get started!




## 2 Pré-requis

### Démarrer Spark


Pour démarrer votre cluster Spark, si cela n'a pas déjà été fait, vous devrez ouvrir un terminal puis exécuter la commande suivante :

Vérifiez que les containers smaster, sworker1 et sworker2 sont démarrés :

### Connexion à Spark

A partir de ce notebook, vous établirez une connexion avec le cluster Spark en python.
Exécutez la cellule ci-dessous pour vous connecter à votre Cluster Spark  :

In [2]:
# N'oubliez pas de fermer la connexion à la fin du TP
# spark.stop()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("spark://smaster:7077").appName("TPDF02").getOrCreate()

Notre objet de connexion 'spark'  dispose d'un ensemble de fonctions pour analyser différentes sources de données : JSON, AVRO, PARQUET, CSV, base de données ...

* `csv(path)`
* `jdbc(url, table, ..., connectionProperties)`
* `json(path)`
* `format(source)`
* `load(path)`
* `orc(path)`
* `parquet(path)`
* `table(tableName)`
* `text(path)`
* `textFile(path)`

On pourra aussi y retrouver des fonctions pour associer des schémas de données à une source de données ou configurer les options de lectures des données :

* `option(key, value)`
* `options(map)`
* `schema(schema)`



## 1 - Les fichiers de données

Pour lire un fichier, il suffit d'utiliser la fonction associée à votre format de fichier:
* csv,txt -> load
* parquet -> parquet
* json -> json 
...

In [3]:
df_csv = spark.read.load("/data/tpspark/sales_info.csv", format="csv", sep=",", inferSchema="true", header="true")

Dans les options de lecture du fichier, nous avons demandé à Spark d'utiliser le séparteur ","  et d'inférer le schéma.

Pour consulter le schéma évaluer par Spark utilisez la fonction printSchema()

In [4]:
df_csv.printSchema()

root
 |-- GOOG: string (nullable = true)
 |-- Sam: string (nullable = true)
 |-- 200: double (nullable = true)



Pour voir les fonctions disponibles avec les dataframes consultez la documentation Spark :
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

Pour avoir un aperçu des données, utilisez la méthode show()

In [5]:
df_csv.show()

+----+-------+-----+
|GOOG|    Sam|  200|
+----+-------+-----+
|GOOG|Charlie|120.0|
|GOOG|  Frank|340.0|
|MSFT|   Tina|600.0|
|MSFT|    Amy|124.0|
|MSFT|Vanessa|243.0|
|  FB|   Carl|870.0|
|  FB|  Sarah|350.0|
|APPL|   John|250.0|
|APPL|  Linda|130.0|
|APPL|   Mike|750.0|
|APPL|  Chris|350.0|
+----+-------+-----+



Vous l'aurez compris Spark utilise un échantillonage des données pour évaluer le schema des données.
Il faut l'avouer, c'est bien plus pratique que les RDD.

Bien entendu, si le schéma ne vous convient pas vous pouvez lui indiquer le schema que vous souhaitez (renommer les noms des colonnes par exemple).

In [6]:
from pyspark.sql.types import *
# Required for StructField, StringType, IntegerType, etc.


fields = [  StructField("Compagnie", StringType(), True),
            StructField("Personne", StringType(), True),
            StructField("Ventes", FloatType(), False)
            ]
            
csvSchema = StructType(fields)

In [8]:
df_csv_with_my_schema = spark.read.option("header", "true").option("delimiter", ",").schema(csvSchema).csv("/data/tpspark/sales_info.csv")

In [9]:
df_csv_with_my_schema.printSchema()

root
 |-- Compagnie: string (nullable = true)
 |-- Personne: string (nullable = true)
 |-- Ventes: float (nullable = true)



In [10]:
df_csv_with_my_schema.show()

+---------+--------+------+
|Compagnie|Personne|Ventes|
+---------+--------+------+
|     GOOG| Charlie| 120.0|
|     GOOG|   Frank| 340.0|
|     MSFT|    Tina| 600.0|
|     MSFT|     Amy| 124.0|
|     MSFT| Vanessa| 243.0|
|       FB|    Carl| 870.0|
|       FB|   Sarah| 350.0|
|     APPL|    John| 250.0|
|     APPL|   Linda| 130.0|
|     APPL|    Mike| 750.0|
|     APPL|   Chris| 350.0|
+---------+--------+------+



Les fichiers csv et texte ne sont pas des formats adaptés pour de grosse volumétrie. 
Une bonne pratique dans les environnements de BIG DATA est de travailler avec des formats binaires pour 2 raisons :
* il consomme moins de stockage.
* il est plus facile de déplacer des données via le réseau.

Dans les écosystèmes HADOOP, les données sont sérialisées sous 2 formes :
* Ligne : Avro, Kryo, Protobuff
* Column : Parquet, Orc

En entreprise, vous trouverez souvent les formats Avro ou parquet pour leur performance.

Pour convertir un fichier en parquet c'est aussi simple que ça :

In [12]:
df_csv_with_my_schema.write.format("parquet").save("//data/tpspark//sales_info.parquet")

In [14]:
!$HADOOP_HOME/bin/hdfs dfs -ls /data/tpspark/sales_info.parquet/

part-00000-662a3bf1-1bab-4666-9fb0-640a67512ba9-c000.snappy.parquet  _SUCCESS


Vous pouvez remarquer qu'au format parquet, notre fichier sales_info.parquet est un répertoire contenant des fichiers part-00000, cela vous rappelle-t'il quelque chose ?
En effet, pour traiter des données parquet sait que les données vont être distribuées et par conséquent applique le principe de partitionnement. Pour un fichier de plusieurs Go, il est possible d'indiquer la taille des partitions parquet.

Bien entendu, la plupart des formats binaires inscrit le schéma des données dans chaque partition.

Vérifions cela :

In [15]:
Sales_parquet = spark.read.parquet("/data/tpspark/sales_info.parquet")

In [16]:
Sales_parquet.printSchema()

root
 |-- Compagnie: string (nullable = true)
 |-- Personne: string (nullable = true)
 |-- Ventes: float (nullable = true)



In [17]:
Sales_parquet.show()

+---------+--------+------+
|Compagnie|Personne|Ventes|
+---------+--------+------+
|     GOOG| Charlie| 120.0|
|     GOOG|   Frank| 340.0|
|     MSFT|    Tina| 600.0|
|     MSFT|     Amy| 124.0|
|     MSFT| Vanessa| 243.0|
|       FB|    Carl| 870.0|
|       FB|   Sarah| 350.0|
|     APPL|    John| 250.0|
|     APPL|   Linda| 130.0|
|     APPL|    Mike| 750.0|
|     APPL|   Chris| 350.0|
+---------+--------+------+



On peut facilement convertir d'un format à un autre : 

In [18]:
Sales_parquet.write.format('json').save('/data/tpspark/sales.json')

Pour lire ensuite vos données.

In [19]:
Sales_json = spark.read.json("/data/tpspark/sales.json")

In [20]:
Sales_json.show()

+---------+--------+------+
|Compagnie|Personne|Ventes|
+---------+--------+------+
|     GOOG| Charlie| 120.0|
|     GOOG|   Frank| 340.0|
|     MSFT|    Tina| 600.0|
|     MSFT|     Amy| 124.0|
|     MSFT| Vanessa| 243.0|
|       FB|    Carl| 870.0|
|       FB|   Sarah| 350.0|
|     APPL|    John| 250.0|
|     APPL|   Linda| 130.0|
|     APPL|    Mike| 750.0|
|     APPL|   Chris| 350.0|
+---------+--------+------+



# 2 Exploiter les DataFrames

Pour manipuler les DataFrames, nous avons un ensemble de méthodes disponibles :

In [21]:
df = spark.read.parquet("/data/tpspark/sales_info.parquet")

In [22]:
dir(df)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collect_as_arrow',
 '_jcols',
 '_jdf',
 '_jmap',
 '_jseq',
 '_lazy_rdd',
 '_repr_html_',
 '_sc',
 '_schema',
 '_sort_cols',
 '_support_repr_html',
 '_to_corrected_pandas_type',
 'agg',
 'alias',
 'approxQuantile',
 'cache',
 'checkpoint',
 'coalesce',
 'colRegex',
 'collect',
 'columns',
 'corr',
 'count',
 'cov',
 'createGlobalTempView',
 'createOrReplaceGlobalTempView',
 'createOrReplaceTempView',
 'createTempView',
 'crossJoin',
 'crosstab',
 'cube',
 'describe',
 'distinct',
 'drop',
 'dropDuplicates',
 'drop_duplicates',
 'dropna',
 'dtypes',
 'exceptAll',
 'explain',
 'fillna',
 'fi

Pour afficher les colonnes :

In [23]:
df.columns

['Compagnie', 'Personne', 'Ventes']

Travaillons à présent sur les manipulations que nous pouvons faire :

In [24]:
dfColVentes=df.select('Ventes')

In [25]:
type(dfColVentes)

pyspark.sql.dataframe.DataFrame

Le résultat retourné par le select est un DataFrame

In [26]:
df.select('Ventes').show()

+------+
|Ventes|
+------+
| 120.0|
| 340.0|
| 600.0|
| 124.0|
| 243.0|
| 870.0|
| 350.0|
| 250.0|
| 130.0|
| 750.0|
| 350.0|
+------+



Pour selectionner les N premières lignes, on trouvera la fonction head :

In [27]:
# Retourne une d'object RowReturns list of Row objects
df.head(2)

[Row(Compagnie='GOOG', Personne='Charlie', Ventes=120.0),
 Row(Compagnie='GOOG', Personne='Frank', Ventes=340.0)]

Pour sélectionner multiple colonne :

In [28]:
df.select(['Compagnie','Ventes']).show()

+---------+------+
|Compagnie|Ventes|
+---------+------+
|     GOOG| 120.0|
|     GOOG| 340.0|
|     MSFT| 600.0|
|     MSFT| 124.0|
|     MSFT| 243.0|
|       FB| 870.0|
|       FB| 350.0|
|     APPL| 250.0|
|     APPL| 130.0|
|     APPL| 750.0|
|     APPL| 350.0|
+---------+------+



ou

In [32]:
df.select(df.Compagnie.alias('test'), df.Ventes).show()

+----+------+
|test|Ventes|
+----+------+
|GOOG| 120.0|
|GOOG| 340.0|
|MSFT| 600.0|
|MSFT| 124.0|
|MSFT| 243.0|
|  FB| 870.0|
|  FB| 350.0|
|APPL| 250.0|
|APPL| 130.0|
|APPL| 750.0|
|APPL| 350.0|
+----+------+



Pour le renommage d'une colonne on peut utiliser la méthode alias :

In [34]:
df.select(df.Compagnie.alias('col1'), df.Ventes.alias('col2')).show()

+----+-----+
|col1| col2|
+----+-----+
|GOOG|120.0|
|GOOG|340.0|
|MSFT|600.0|
|MSFT|124.0|
|MSFT|243.0|
|  FB|870.0|
|  FB|350.0|
|APPL|250.0|
|APPL|130.0|
|APPL|750.0|
|APPL|350.0|
+----+-----+



Si on veut créer de nouvelles colonnes à partir d'une colonne existante  :

In [30]:
df.withColumn('VentesAdd',df['Ventes'] * 10).show()

+---------+--------+------+---------+
|Compagnie|Personne|Ventes|VentesAdd|
+---------+--------+------+---------+
|     GOOG| Charlie| 120.0|   1200.0|
|     GOOG|   Frank| 340.0|   3400.0|
|     MSFT|    Tina| 600.0|   6000.0|
|     MSFT|     Amy| 124.0|   1240.0|
|     MSFT| Vanessa| 243.0|   2430.0|
|       FB|    Carl| 870.0|   8700.0|
|       FB|   Sarah| 350.0|   3500.0|
|     APPL|    John| 250.0|   2500.0|
|     APPL|   Linda| 130.0|   1300.0|
|     APPL|    Mike| 750.0|   7500.0|
|     APPL|   Chris| 350.0|   3500.0|
+---------+--------+------+---------+



Pour renommer une colonne

In [31]:
# Simple Rename
df.withColumnRenamed('Compagnie','Company').show()

+-------+--------+------+
|Company|Personne|Ventes|
+-------+--------+------+
|   GOOG| Charlie| 120.0|
|   GOOG|   Frank| 340.0|
|   MSFT|    Tina| 600.0|
|   MSFT|     Amy| 124.0|
|   MSFT| Vanessa| 243.0|
|     FB|    Carl| 870.0|
|     FB|   Sarah| 350.0|
|   APPL|    John| 250.0|
|   APPL|   Linda| 130.0|
|   APPL|    Mike| 750.0|
|   APPL|   Chris| 350.0|
+-------+--------+------+



On peut filtrer sur un critére :

In [ ]:
df.filter(df.Personne=="Sam").show()

Pour les opérations d'agrégation, on utilisera les fonctions du module pyspark.sql.functions lequel regroupe les fonctions max, min, mean ...

In [ ]:
import pyspark.sql.functions as f

new_df.groupBy('Company').agg(f.max('Vente').alias('Max Vente'), f.min('Vente').alias('Min Vente'), f.mean('Vente').alias('Moy_Vente')).orderBy('Company').show()

Comme pour les bases de données relationnelles, on peut faire des jointures entre des sources de données avec différents formats :

In [ ]:
from pyspark.sql import *
# Création d'un dataframe des localisations des différentes compagnies.
# Le code ci-dessous indique comment créer un dataframe manuellement

Location = Row("Compagnie", "location")
location1 = Location('GOOG', 'US')
location2 = Location('MSFT', 'EUROPE')
location3 = Location('FB', 'ASIA')

locationRows =[location1,location2,location3]
df_location = spark.createDataFrame(locationRows)
type(df_location)

On affiche le contenu de notre nouveau dataframe :

In [ ]:
df_location.show()

On affiche la structure du dataframe :

In [ ]:
df.printSchema()

On réalise la jointure entre le dataframe df listant toutes les ventes des companies et le dataframe df_location précisant la localisation des companies :

In [ ]:
df.join(df_location, df_location.Compagnie == df.Compagnie, "left_outer").show()

### Utilisation du SQL

Pour utiliser des requêtes SQL directement sur un DataFrame, vous devrez l'enregistrer dans une vue temporaire:

In [35]:
# La méthode createOrReplaceTempView enregistre 
# le DataFrame comme une view temporaire
df.createOrReplaceTempView("sales")

In [36]:
sql_sales = spark.sql("SELECT * FROM sales")

In [37]:
sql_sales

DataFrame[Compagnie: string, Personne: string, Ventes: float]

In [38]:
sql_sales.show()

+---------+--------+------+
|Compagnie|Personne|Ventes|
+---------+--------+------+
|     GOOG| Charlie| 120.0|
|     GOOG|   Frank| 340.0|
|     MSFT|    Tina| 600.0|
|     MSFT|     Amy| 124.0|
|     MSFT| Vanessa| 243.0|
|       FB|    Carl| 870.0|
|       FB|   Sarah| 350.0|
|     APPL|    John| 250.0|
|     APPL|   Linda| 130.0|
|     APPL|    Mike| 750.0|
|     APPL|   Chris| 350.0|
+---------+--------+------+



Bien entendu, vous pouvez appliquer des requêtes SQL avec des conditions :

In [39]:
spark.sql("SELECT * FROM sales WHERE Ventes > 500").show()

+---------+--------+------+
|Compagnie|Personne|Ventes|
+---------+--------+------+
|     MSFT|    Tina| 600.0|
|       FB|    Carl| 870.0|
|     APPL|    Mike| 750.0|
+---------+--------+------+



Spark SQL supporte un sous ensemble de la norme SQL92. 

Bravo !!!

Vous êtes prêt à faire vos premiers exercices avec Spark.